# Assignment administration

- Run the following cell.
- Follow the instructions below.

In [ ]:
%maven commons-io:commons-io:2.6
import org.apache.commons.io.FileUtils;
import org.apache.commons.io.filefilter.TrueFileFilter;

void createAssignment(String assignmentName, int numberOfExercises) throws Exception {
    if(assignmentName == null || assignmentName.isEmpty()){
        throw new Exception("No assignment name");
    }
    
    File templateDirectory = new File("/home/nbgadmin/templates/");
    File sourceDirectory = new File("/home/nbgadmin/source/");
    File assignmentDirectory = new File(sourceDirectory, assignmentName);
    if (!templateDirectory.exists() || !templateDirectory.canRead()) {
        throw new Exception("Can not read: " + templateDirectory.getPath());
    }
    if (!sourceDirectory.exists()) {
        sourceDirectory.mkdirs();
    }
    if (!sourceDirectory.canWrite()) {
        throw new Exception("Can not write: " + sourceDirectory.getPath());
    }
    if (assignmentDirectory.exists()) {
        throw new Exception("Already existing: " + assignmentDirectory.getPath());
    }
    
    for (File file : templateDirectory.listFiles(new FilenameFilter() {
        public boolean accept(File directory, String name) {
            if (name.endsWith(".ipynb")) {
                return true;
            } else {
                return false;
            }
        }
    })) {
        if (file.getName().equals("AssignmentTemplate.ipynb")) {
            for (int i = 1; i <= numberOfExercises; i++) {
                File newFile = new File(assignmentDirectory, "Exercise-" + i + ".ipynb");
                FileUtils.copyFile(file, newFile);
                System.out.println("Created file: " + newFile.getPath());
            }
        }
        File newFile = new File(assignmentDirectory, file.getName());
        FileUtils.copyFile(file, newFile);
        System.out.println("Created file: " + newFile.getPath());
    }
}

void copyFeedback(String assignmentName) throws Exception {
    if(assignmentName == null || assignmentName.isEmpty()){
        throw new Exception("No assignment name");
    }
    
    File feedbackBaseDirectory = new File("/home/nbgadmin/feedback/");
    if (!feedbackBaseDirectory.exists() || !feedbackBaseDirectory.canRead()) {
        throw new Exception("Can not read: " + feedbackBaseDirectory.getPath());
    }
    
    int copies = 0;
    int directories = 0;
    for (File feedbackUserDirectory : feedbackBaseDirectory.listFiles()) {
        // Should only contain directories, just skip
        if(!feedbackUserDirectory.isDirectory()) {
            continue;
        }
        File feedbackAssignmentDirectory = new File(feedbackUserDirectory, assignmentName);
        // User has not participated, just skip
        if (!feedbackAssignmentDirectory.exists()) {
            continue;
        }
        
        directories++;
        
        if (!feedbackAssignmentDirectory.canRead()) {
            System.err.println("Can not read feedback for user, skipping: " + feedbackAssignmentDirectory.getPath());
            continue;
        }
        File userHomeDirectory = new File("/home/" + feedbackUserDirectory.getName());
        if (!userHomeDirectory.exists()) {
            System.err.println("User home not found, skipping: " + userHomeDirectory.getPath());
            continue;
        }
        if (!userHomeDirectory.canWrite()) {
            System.err.println("Can not write user home, skipping: " + userHomeDirectory.getPath());
            continue;
        }
        File resultsDirectory = new File(userHomeDirectory, "results");
        if (resultsDirectory.exists() && !resultsDirectory.canWrite()) {
            System.err.println("Can not write results: " + resultsDirectory.getPath());
            continue;
        }
        File assignmentResultsDirectory = new File(resultsDirectory, assignmentName);

		try {
			FileUtils.copyDirectory(feedbackAssignmentDirectory, assignmentResultsDirectory);
		} catch (IOException e) {
            System.err.println("Error copying: " + e.getMessage());
            continue;
		}
        System.out.println("Provided feedback: " + assignmentResultsDirectory);
        copies++;
    }
    System.out.println(directories + " feedback entries found.");
    System.out.println(copies + " feedback entries copied.");
}

void printAutogradeCommands(String assignmentName) {
    System.out.println("cd \"/home/nbgadmin\"");
    System.out.println("nbgrader autograde \"" + assignmentName + "\"");
}

void printFeedbackCommands(String assignmentName) {
    System.out.println("cd \"/home/nbgadmin\"");
    System.out.println("nbgrader feedback \"" + assignmentName + "\"");
}

## Configuration

Specify an **assignment name** for the next steps.


In [ ]:
String assignmentName = "Example";

## Create an assignment

1. Specify the **number of exercise files to create** in the following cell.
2. Run the createAssignment cell. It will create the assignment directory and template files.

In [ ]:
int numberOfExercisesFilesToCreate = 3;

In [ ]:
createAssignment(assignmentName, numberOfExercisesFilesToCreate);

3. Open the [source directory in the files view](/user/nbgadmin/tree/source).  
  * Edit the files.  
  * **Delete** unused files.  
  * Set the points of the **autograder test cells** or remove the cells.

## Release the assignment

Open [Formgrader](/user/nbgadmin/formgrader).

1. Click on [**Generate**].  
   (Generated files will be stored in the _release_ directory.)
2. Click on [**Preview**].  
   Check the generated files.  
   If there are errors, edit the files and continue with step 1.
3. Click on [**Release**].  
   If you want to test the release, you can login as _nbgtesta_, _nbgtestb_, and _nbgtestc_.
4. You may want to make a backup of the _source/[assignmentName]_ directory to save your solution.
5. **Inform your students** about the assignment and a **deadline**.

## Finish the assignment

Open [Formgrader](/user/nbgadmin/formgrader).

1. Click on [**Collect**].  
   (Collected files will be stored in the _submitted_ directory.)
2. Click on [**Release**] to to end the release.
3. You could stop all running servers at the [hub admin](/hub/admin) (which also will affect this notebook).
4. You may want to make a backup of the _submitted_ directory to save the student solutions.

## Rate the assignment

1. Open a [terminal](/user/nbgadmin/terminals/1) and use the lines printed in the following cell to **autograde** the assignment.
2. Open [Formgrader](/user/nbgadmin/formgrader) and click on **[Manual Grading]** and rate the answers.
3. You may want to make a backup of the nbgrader database _gradebook.db_ to save the students points.

In [ ]:
printAutogradeCommands(assignmentName);

## Provide feedback

1. Open a [terminal](/user/nbgadmin/terminals/1) and use the lines printed in the following cell to create **feedback** of the assignment.
2. Run the cell afterwards to generate feedback in HTML format and to copy the generated feedback to the students filesystem directories.

In [ ]:
printFeedbackCommands(assignmentName);

In [ ]:
copyFeedback(assignmentName);